In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [73]:
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction.settings import MinimalFCParameters
from tsfresh.transformers import RelevantFeatureAugmenter

In [71]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import xgboost as xgb

In [4]:
stu_vle = pd.read_csv('data/stvl_ccc14b.csv')
stu_as = pd.read_csv('data/stas_ccc14b.csv')
ass = pd.read_csv('data/ass_ccc14b.csv')
vle = pd.read_csv('data/vle_ccc14b.csv')
stu_info = pd.read_csv('data/stuinfo_ccc14b.csv')
stu_reg = pd.read_csv('data/stureg_ccc14b.csv')

# Preprocessing

### Drop early withdrawal students

Or rather, determine which students to keep.

In [5]:
stu_reg.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,CCC,2014B,28418,-37.0,NaN
1,CCC,2014B,29764,-34.0,NaN
2,CCC,2014B,29820,-57.0,NaN
3,CCC,2014B,40333,-30.0,17.0
4,CCC,2014B,40604,-17.0,NaN


In [6]:
pop_of_interest = stu_reg.drop(stu_reg[stu_reg.date_unregistration <= 67].index)['id_student'].unique()

In [7]:
len(stu_vle.drop(stu_vle[~stu_vle.id_student.isin(pop_of_interest)].index)['id_student'].unique()) # num students left in course who clicked

1300

In [12]:
clkd_pop_of_interest = stu_vle.drop(stu_vle[~stu_vle.id_student.isin(pop_of_interest)].index)['id_student'].unique()

### Make y (targets) column

In [10]:
stu_info.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,CCC,2014B,28418,F,West Midlands Region,A Level or Equivalent,20-30%,0-35,0,30,N,Fail
1,CCC,2014B,29764,M,East Anglian Region,A Level or Equivalent,50-60%,0-35,0,90,N,Distinction
2,CCC,2014B,29820,M,East Anglian Region,HE Qualification,40-50%,0-35,0,60,N,Pass
3,CCC,2014B,40333,M,North Region,HE Qualification,0-10%,35-55,0,30,N,Withdrawn
4,CCC,2014B,40604,M,Ireland,A Level or Equivalent,NaN,35-55,0,30,N,Pass


In [13]:
y_col = stu_info.drop(stu_info[~stu_info.id_student.isin(clkd_pop_of_interest)].index)

In [14]:
y_col = y_col.drop(['code_module', 'code_presentation', 'gender', 'region', 'highest_education', 'imd_band', 'age_band', 'num_of_prev_attempts', 'studied_credits', 'disability'], axis=1)
y_col.final_result.replace(to_replace=dict(Pass=1, Distinction=1, Fail=0, Withdrawn=0), inplace=True)
y_col = y_col.sort_values(by=['id_student'])

In [15]:
y_col.head()

,id_student,final_result
0,28418,0
1,29764,1
2,29820,1
4,40604,1
5,42638,1


In [79]:
y_col = y_col.set_index('id_student')

In [82]:
y = y_col['final_result']

### Train-test split BEFORE feature-extraction

Very important or else this would perform significantly worse in production.

In [83]:
y_train, y_test = train_test_split(y, test_size=0.2)

In [88]:
len(y_train), len(y_test)

(1040, 260)

### Make students click stream data (from which to generate features)

In [91]:
df_train = stu_vle.loc[stu_vle.id_student.isin(y_train.index)]
df_test = stu_vle.loc[stu_vle.id_student.isin(y_test.index)]

In [92]:
df_train = df_train.drop(['code_module', 'code_presentation', 'id_site'], axis=1)
df_train = df_train.groupby(['id_student', 'date']).sum() #sum daily clicks per student
df_train = df_train.reset_index()

In [94]:
df_test = df_test.drop(['code_module', 'code_presentation', 'id_site'], axis=1)
df_test = df_test.groupby(['id_student', 'date']).sum() #sum daily clicks per student
df_test = df_test.reset_index()

In [99]:
df_train.shape, df_test.shape

((66932, 3), (15815, 3))

### Make X input for tsfresh feature augmenter (just student IDs of each group)

In [100]:
X_train = pd.DataFrame(index=y_train.index)
X_test = pd.DataFrame(index=y_test.index)

In [103]:
X_train.head()

""
id_student
147675
601949
251385
2016517
383158


# Create and run pipeline with feature extraction and classifier

This all may look strange but setting the parameters like this is precisely how the ts docs outline, for the use case of extracting features on the train set, and then only using those features when evaluating the test set. (that is, NOT extracting features from the test set) (see docs at https://github.com/blue-yonder/tsfresh/blob/master/notebooks/pipeline_with_two_datasets.ipynb)

In [104]:
ppl = Pipeline([('fresh', RelevantFeatureAugmenter(column_id='id_student', column_sort='date', 
                                                   default_fc_parameters=MinimalFCParameters())),
                ('clf', RandomForestClassifier())])

In [105]:
# for the fit on the train test set, we set the fresh__timeseries_container to `df_train`
ppl.set_params(fresh__timeseries_container=df_train)
ppl.fit(X_train, y_train)

Feature Extraction: 100%|██████████| 20/20 [00:00<00:00, 257.30it/s]
//anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('fresh',
                 RelevantFeatureAugmenter(chunksize=None,
                                          column_id='id_student',
                                          column_kind=None, column_sort='date',
                                          column_value=None,
                                          default_fc_parameters={'length': None,
                                                                 'maximum': None,
                                                                 'mean': None,
                                                                 'median': None,
                                                                 'minimum': None,
                                                                 'standard_deviation': None,
                                                                 'sum_values': None,
                                                                 'variance': None},
                    

In [106]:
# for the predict on the test test set, we set the fresh__timeseries_container to `df_test`
ppl.set_params(fresh__timeseries_container=df_test)
y_pred = ppl.predict(X_test)

Feature Extraction: 100%|██████████| 20/20 [00:00<00:00, 782.50it/s]


In [107]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.79      0.74      0.76       131
           1       0.75      0.80      0.77       129

    accuracy                           0.77       260
   macro avg       0.77      0.77      0.77       260
weighted avg       0.77      0.77      0.77       260

